In [1]:
import torch
import torchvision
from torch.export import export

resnet18 = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
sample_input = (torch.randn(4, 3, 224, 224), )
exported = export(resnet18, sample_input)
print(exported)

ExportedProgram:
    class GraphModule(torch.nn.Module):
        def forward(self, p_conv1_weight: "f32[64, 3, 7, 7]", p_bn1_weight: "f32[64]", p_bn1_bias: "f32[64]", p_getattr_l__self___layer1___0___conv1_weight: "f32[64, 64, 3, 3]", p_getattr_l__self___layer1___0___bn1_weight: "f32[64]", p_getattr_l__self___layer1___0___bn1_bias: "f32[64]", p_getattr_l__self___layer1___0___conv2_weight: "f32[64, 64, 3, 3]", p_getattr_l__self___layer1___0___bn2_weight: "f32[64]", p_getattr_l__self___layer1___0___bn2_bias: "f32[64]", p_getattr_l__self___layer1___1___conv1_weight: "f32[64, 64, 3, 3]", p_getattr_l__self___layer1___1___bn1_weight: "f32[64]", p_getattr_l__self___layer1___1___bn1_bias: "f32[64]", p_getattr_l__self___layer1___1___conv2_weight: "f32[64, 64, 3, 3]", p_getattr_l__self___layer1___1___bn2_weight: "f32[64]", p_getattr_l__self___layer1___1___bn2_bias: "f32[64]", p_getattr_l__self___layer2___0___conv1_weight: "f32[128, 64, 3, 3]", p_getattr_l__self___layer2___0___bn1_weight: "f32[12

In [2]:
from torch_xla.stablehlo import exported_program_to_stablehlo

stablehlo_program = exported_program_to_stablehlo(exported)
print(stablehlo_program.get_stablehlo_text('forward')[0:4000],"\n...")

ImportError: /home/douliyang/large/mlir-workspace/iree/iree/lib/python3.11/site-packages/_XLAC.cpython-311-x86_64-linux-gnu.so: undefined symbol: _ZN3c109TupleTypeC1ESt6vectorINS_4Type24SingletonOrSharedTypePtrIS2_EESaIS4_EENS_8optionalINS_13QualifiedNameEEESt10shared_ptrINS_14FunctionSchemaEE

In [2]:
import os
import timeit
import torch
import torchvision
import torch_xla.core.xla_model as xm
from torch.export import export
from torch_xla.stablehlo import exported_program_to_stablehlo

os.environ["PJRT_DEVICE"] = "GPU"
os.environ["GPU_NUM_DEVICES"] = "1"

def measure_perf(fn, *args, n_iters=10, **kwargs):
    times = []
    for i in range(n_iters):
        start_t = timeit.default_timer()
        result = fn(*args, **kwargs)
        end_t = timeit.default_timer()
        times.append(end_t - start_t)
    return times

# Benchmark only one model to isolate the issue
model_class = torchvision.models.resnet18
model = model_class()
input_data = torch.randn(128, 3, 224, 224)

results = {}

with torch.inference_mode():

    xla_cuda = xm.xla_device(devkind="GPU")
    results["CUDA (XLA)"] = measure_perf(model.to(xla_cuda), input_data.to(xla_cuda))
    exported = export(model.to(xla_cuda), (input_data.to(xla_cuda), ))
    model_stablehlo = exported_program_to_stablehlo(exported)
    results["CUDA (XLA, StableHLO)"] = measure_perf(model_stablehlo, input_data.to(xla_cuda))

print(results)

/home/douliyang/large/mlir-workspace/iree/iree/lib/python3.11/site-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


RuntimeError: torch_xla/csrc/runtime/pjrt_registry.cc:214 : Check failed: client 
*** Begin stack trace ***
	tsl::CurrentStackTrace()
	torch_xla::runtime::InitializePjRt(std::string const&)
	torch_xla::runtime::PjRtComputationClient::PjRtComputationClient()
	
	torch_xla::runtime::GetComputationClient()
	torch_xla::bridge::GetDefaultDevice()
	torch_xla::bridge::GetCurrentDevice()
	torch_xla::bridge::GetCurrentAtenDevice()
	
	
	
	_PyObject_MakeTpCall
	_PyEval_EvalFrameDefault
	
	PyEval_EvalCode
	
	_PyEval_EvalFrameDefault
	
	_PyEval_EvalFrameDefault
	
	_PyEval_EvalFrameDefault
	
	
	
	PyObject_Vectorcall
	_PyEval_EvalFrameDefault
	
	
	PyObject_Call
	_PyEval_EvalFrameDefault
	
	_PyEval_EvalFrameDefault
	
	_PyEval_EvalFrameDefault
	
	_PyEval_EvalFrameDefault
	
	_PyEval_EvalFrameDefault
	
	_PyEval_EvalFrameDefault
	
	
	
	
	
	
	_PyEval_EvalFrameDefault
	
	PyEval_EvalCode
	
	
	PyObject_Vectorcall
	_PyEval_EvalFrameDefault
	_PyFunction_Vectorcall
	
	Py_RunMain
	Py_BytesMain
	
	__libc_start_main